# Load SardiNIA Database

In [1]:
library(readr)

In [2]:
full_db <- read_tsv( '20171017_SardiNIA_WaveIV_only.txt')

Parsed with column specification:
cols(
  .default = col_double(),
  id_individual = col_integer(),
  Wave = col_integer(),
  FirstVisitDate = col_date(format = ""),
  SecondVisitDate = col_date(format = ""),
  ThirdVisitDate = col_date(format = ""),
  FourthVisitDate = col_date(format = ""),
  labsHbA1Cdx = col_character(),
  labsG6PD = col_character(),
  labsZnPP = col_character(),
  labsBilirubinad = col_character(),
  labsBilirubinat = col_character(),
  labsSodiemia = col_character(),
  labsPotassiemia = col_character(),
  labsPCR = col_character(),
  labsTie = col_character(),
  disMIname = col_character(),
  disMIwhen = col_date(format = ""),
  disAPname = col_character(),
  disAPwhen = col_date(format = ""),
  disHFname = col_character()
  # ... with 192 more columns
)
See spec(...) for full column specifications.


In [3]:
# print # rows, # cols
dim(full_db)

[1] 4705 1286

In [4]:
head(full_db)

id_individual,id_sir,id_mad,Wave,Visit,Age,Sex,Education,Occupation,MaritalStatus,⋯,SphPWVSecondM_DT_DIST,SphPWVSecondM_N_Measurements,SphPWVSecondM_PP_DEVIATION,SphPWVSecondM_PP_MDT,SphPWVSecondM_PWV,SphPWVSecondM_PWVERR,SphPWVSecondM_PWV_DIST,SphPWVSecondM_PX_DIST,SphPWVSecondM_SP,SphPWVSecondM_date
2,1573,1,4,3,36.8,1,5,71,1,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
7,6756,16525,4,4,72.8,1,3,92,2,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,7,8546,4,3,43.6,1,5,140,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
10,7,8546,4,3,37.3,0,4,72,3,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12,7,8546,4,1,22.9,0,4,97,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,16137,15362,4,4,76.3,1,2,70,1,⋯,570,3,4.0651,62.90355,7.708743,0.5383492,480,90,106,2015-12-01


# Get the trait names for the OLD SphygmoCor machine

Grab all the column names starting with the prefix "SphfirstM_"

In [5]:
library(stringr)

In [6]:
old_names <- sort( str_subset( names(full_db), "SphfirstM_" ) )

In [7]:
head(old_names)

[1] "SphfirstM_C_AGPH"      "SphfirstM_C_AGPH_HR75" "SphfirstM_C_AI"       
[4] "SphfirstM_C_AP"        "SphfirstM_C_AP_HR75"   "SphfirstM_C_DD"

Drop the prefix "SphfirstM_" from the variable name by taking the substring 11 characters from the beginning of the name.

In [8]:
old_names <- sort( sapply( old_names, function(x) str_sub( x, 11), USE.NAMES=F ) )

In [9]:
head( old_names )

[1] "C_AGPH"      "C_AGPH_HR75" "C_AI"        "C_AP"        "C_AP_HR75"  
[6] "C_DD"

# Get the trait names for the NEW SphygmoCor machine

Grab all the column names starting with the prefix "SphfirstM_"

In [10]:
new_names <- sort( str_subset( names(full_db), "SphPWAFirstM_" ) )

In [11]:
length( new_names )

[1] 54

In [12]:
head(new_names)

[1] "SphPWAFirstM_C_AGPH" "SphPWAFirstM_C_AI"   "SphPWAFirstM_C_AL"  
[4] "SphPWAFirstM_C_AP"   "SphPWAFirstM_C_ATI"  "SphPWAFirstM_C_DD"

In [13]:
new_names <- sort( sapply( new_names, function(x) str_sub( x, 14), USE.NAMES=F ) )

In [14]:
head( new_names )

[1] "C_AGPH" "C_AI"   "C_AL"   "C_AP"   "C_ATI"  "C_DD"

# Get the intersection of traits names measured by BOTH new and old machines

In [15]:
intersection <- sort( intersect( new_names, old_names ) )

In [16]:
intersection

[1] "C_AGPH"      "C_AI"        "C_AP"        "C_DD"        "C_DD_PERIOD"
 [6] "C_DP"        "C_DTI"       "C_ED_PERIOD" "C_ESP"       "C_MEANP"    
[11] "C_P1"        "C_P1_HEIGHT" "C_P2"        "C_PERIOD"    "C_PH"       
[16] "C_SP"        "C_SVI"       "C_T1"        "C_T1ED"      "C_T2"       
[21] "C_T2ED"      "C_TTI"       "DP"          "ED"          "HR"         
[26] "P_AI"        "P_DP"        "P_ESP"       "P_MAX_DPDT"  "P_MEANP"    
[31] "P_P1"        "P_P2"        "P_SP"        "P_T1"        "P_T1ED"     
[36] "P_T2"        "P_T2ED"      "QUALITY_ED"  "SP"

# For each trait, do a pairwise t-test

In [17]:
library(tidyr)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [18]:
n_vars <- length(intersection)
# initialize empty results table
results <- tibble( name=intersection,
                   n_obs=rep(NA_integer_, n_vars), #n_obs=rep(NA_integer_, n_vars),
                   old_mean=rep('', n_vars),       #old_mean=rep(NA_real_, n_vars),
                   new_mean=rep('', n_vars),       #new_mean=rep(NA_real_, n_vars),
                   diff=rep('', n_vars),           #diff=rep(NA_real_, n_vars),
                   old_std=rep('', n_vars),        #old_std=rep(NA_real_, n_vars),
                   new_std=rep('', n_vars),        
                   p_value=rep('', n_vars),
                 )
for( name in intersection ){
    old_col_name <- paste0( 'SphfirstM_', name )
    new_col_name <- paste0( 'SphPWAFirstM_', name )
    subset <- full_db %>% select( id_individual, old_col_name, new_col_name ) %>% drop_na()
    old <- subset[[old_col_name]]
    new <- subset[[new_col_name]]
    test_result <- t.test( old, new, paired=T )
    results[ results$name == name, -1] <- c( 
        length(old),
        sprintf( "%0.1f", mean(old) ), 
        sprintf( "%0.1f", mean(new) ), 
        sprintf( "%0.1f", test_result$estimate ), 
        sprintf( "%0.1f", sd(old) ),
        sprintf( "%0.1f", sd(new) ), 
        sprintf( "%0.3f", test_result$p.value ) )
}

# Results

In [19]:
results

name,n_obs,old_mean,new_mean,diff,old_std,new_std,p_value
C_AGPH,73,23.9,26.0,-2.2,13.8,14.7,0.022
C_AI,73,136.0,141.9,-5.9,26.5,31.5,0.017
C_AP,73,8.3,10.4,-2.1,6.0,7.8,0.000
C_DD,73,574.9,599.2,-24.4,127.0,132.7,0.042
C_DD_PERIOD,73,63.8,64.3,-0.5,4.1,3.8,0.195
C_DP,73,74.0,77.6,-3.7,10.1,11.2,0.002
C_DTI,73,3211.4,3403.9,-192.5,504.9,533.1,0.000
C_ED_PERIOD,73,36.2,35.7,0.5,4.2,3.8,0.162
C_ESP,73,98.1,104.5,-6.4,14.5,15.9,0.000
C_MEANP,73,88.8,93.7,-5.0,12.0,12.8,0.000
